In [5]:
import pandas as pd
import numpy as np
from dash import Dash, html, dcc
import plotly.express as px
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
df = pd.read_csv("./test_final.csv")
df = df.iloc[: , 1:]

In [14]:
df.head()

,ID,Year,First Name,Middle Name,Last Name,party,Level,Position,state,race_ethnicity,state_name,ideology,leadership
0,196595k,2022,Alma,S.,Adams,Democrat,Congress,U.S. Representative,NC,Black,North Carolina,0.221420,0.736368
1,627442k,2022,Cindy,NaN,Axne,Democrat,Congress,U.S. Representative,IA,White,Iowa,0.508275,0.465520
2,226476k,2022,Karen,R.,Bass,Democrat,Congress,U.S. Representative,CA,Black,California,0.153370,0.849639
3,197852k,2022,Joyce,B.,Beatty,Democrat,Congress,U.S. Representative,OH,Black,Ohio,0.254428,0.639839
4,495403k,2022,Stephanie,NaN,Bice,Republican,Congress,U.S. Representative,OK,White,Oklahoma,NaN,NaN


In [63]:
state_party_df = df[['party', 'state']]
#democrats = 1, republicans = 2
state_party_df['party'].replace(['Democrat', 'Republican'],
                        [1, 2], inplace=True)
party_scores = state_party_df.groupby('state').mean()
print('number of states with at least one female politician:', len(party_scores))

number of states with at least one female politician: 40


In [ ]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd
import plotly.figure_factory as ff

app = Dash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options

fig = px.bar(df, x="Last Name", y="ideology", color="party", barmode="group")


app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Feb/2022 15:38:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 15:38:20] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_2_0m1645908467.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 15:38:20] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_2_0m1645908467.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 15:38:20] "GET /_dash-component-suites/dash/deps/react@16.v2_2_0m1645908467.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 15:38:20] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_2_0m1645908467.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 15:38:20] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_2_0m1645908467.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 15:38:20] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_2_0m1645908467.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 15:38:20] "GET /_dash-component-s

In [12]:
import plotly.figure_factory as ff
df['state'] = df_sample['state'].apply(lambda x: str(x).zfill(2))
df_sample['FIPS'] = df_sample['State FIPS Code']

colorscale = ["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]
endpts = list(np.linspace(1, 12, len(colorscale) - 1))
fips = df_sample['FIPS'].tolist()
values = df_sample['Unemployment Rate (%)'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values,
    binning_endpoints=endpts,
    colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='USA by Unemployment %',
    legend_title='% unemployed'
)

fig.layout.template = None
fig.show()

In [32]:
# Get this figure: fig = py.get_figure("https://plotly.com/~amaras3/18/")
# Get this figure's data: data = py.get_figure("https://plotly.com/~amaras3/18/").get_data()
# Add data to this figure: py.plot(Data([Scatter(x=[1, 2], y=[2, 3])]), filename ="Pace", fileopt="extend")
# Get z data of first trace: z1 = py.get_figure("https://plotly.com/~amaras3/18/").get_data()[0]["z"]

# Get figure documentation: https://plotly.com/python/get-requests/
# Add data documentation: https://plotly.com/python/file-options/

# If you're using unicode in your file, you may need to specify the encoding.
# You can reproduce this figure in Python with the following code!

# Learn about API authentication here: https://plotly.com/python/getting-started
# Find your api_key here: https://plotly.com/settings/api

import plotly as py
from plotly.graph_objs import *
trace1 = {
  "name": "Republican", 
  "type": "choropleth", 
  "zmax": 1, 
  "zmin": 0, 
  "z": ["1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1"], 
  "zauto": False, 
  "text": ["Alabama", "Alaska", "Arizona", "Arkansas", "Florida", "Georgia", "Idaho", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Michigan", "Mississippi", "Missouri", "Montana", "Nebraska", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Pennsylvania", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "West Virginia", "Wisconsin", "Wyoming"], 
  "hoverinfo": "text", 
  "showscale": False, 
  "locationmode": "USA-states", 
  "locations": ["AL", "AK", "AZ", "AR", "FL", "GA", "ID", "IN", "IA", "KS", "KY", "LA", "MI", "MS", "MO", "MT", "NE", "NC", "ND", "OH", "OK", "PA", "SC", "SD", "TN", "TX", "UT", "WV", "WI", "WY"], 
  "autocolorscale": False
}
trace2 = {
  "name": "Democrat", 
  "type": "choropleth", 
  "zmax": 1, 
  "zmin": 0, 
  "z": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 
  "zauto": False, 
  "text": ["California", "Colorado", "Connecticut", "Delaware", "District of Columbia", "Hawaii", "Illinois", "Maine", "Maryland", "Massachusetts", "Minnesota", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "Oregon", "Rhode Island", "Vermont", "Virginia", "Washington"], 
  "hoverinfo": "text", 
  "showscale": False, 
  "locationmode": "USA-states", 
  "locations": ["CA", "CO", "CT", "DE", "DOC", "HI", "IL", "ME", "MD", "MA", "MN", "NV", "NH", "NJ", "NM", "NY", "OR", "RI", "VT", "VA", "WA"], 
  "autocolorscale": False
}
data = Data([trace1, trace2])
layout = {
  "geo": {
    "scope": "usa", 
    "lonaxis": {
      "range": [-180, -50], 
      "showgrid": False, 
      "gridwidth": 1.5999999999999999
    }, 
    "showland": True, 
    "lakecolor": "rgb(255, 255, 255)", 
    "landcolor": "rgba(237, 247, 138, 0.28)", 
    "projection": {"type": "albers usa"}, 
    "showrivers": False, 
    "countrycolor": "rgb(102, 102, 102)", 
    "countrywidth": 0.1, 
    "showsubunits": True, 
    "subunitcolor": "rgb(102, 102, 102)", 
    "subunitwidth": 0.5
  }, 
  "title": "<b>Map of Election Results</b>", 
  "width": 800, 
  "margin": {
    "b": 100, 
    "l": 0, 
    "r": 50, 
    "t": 100, 
    "pad": 4
  }, 
  "autosize": False, 
  "hovermode": "closest", 
  "showlegend": True
}
fig = Figure(data=data, layout=layout)
fig.show()